In [5]:
import os
import multiprocessing
import time
from parallel_task_classes import *

if __name__ == '__main__':

    # Define parameters
    layout_name = "38.png"  
    layout_path = os.path.join( os.getcwd(), 'images', layout_name)
    
    # Establish communication queues.
    cell_queue_layout = multiprocessing.Queue()
    partial_cell_queue_layout = multiprocessing.Queue()
    sorted_cell_queue_layout = multiprocessing.Queue()
    

    extractor_done_event_layout = multiprocessing.Event()
    merger_done_event_layout = multiprocessing.Event()
    collector_done_event_layout = multiprocessing.Event()
    row_done_event_layout = multiprocessing.Event()
    row_event_collector_validator_layout = multiprocessing.Event()
    
    pipe_from_extractor_head_layout, pipe_from_extractor_tail_layout = multiprocessing.Pipe()

    pipe_from_collector_head_layout, pipe_from_collector_tail_layout = multiprocessing.Pipe()

    row_event_collector_validator_layout = multiprocessing.Event()
    
    type_name = '_layout'
    cell_extractor_layout = Cell_extractor(type_name, cell_queue_layout, partial_cell_queue_layout, layout_path, 
                                           extractor_done_event_layout, row_done_event_layout, pipe_from_extractor_head_layout)
    cell_collector_layout = Cell_collector(type_name, cell_queue_layout, sorted_cell_queue_layout,
                                           extractor_done_event_layout, merger_done_event_layout, 
                                           row_done_event_layout, pipe_from_extractor_tail_layout, collector_done_event_layout, 
                                           pipe_from_collector_head_layout, row_event_collector_validator_layout)
    cell_merger_layout = Cell_merger(type_name, partial_cell_queue_layout, cell_queue_layout, 
                                     extractor_done_event_layout, merger_done_event_layout)
    

    SEM_name = "37.png" 
    SEM_path = os.path.join( os.getcwd(), 'images', SEM_name)
    
    # Establish communication queues.
    cell_queue_SEM = multiprocessing.Queue()
    partial_cell_queue_SEM = multiprocessing.Queue()
    sorted_cell_queue_SEM = multiprocessing.Queue()

    extractor_done_event_SEM = multiprocessing.Event()
    merger_done_event_SEM = multiprocessing.Event()
    collector_done_event_SEM = multiprocessing.Event()
    row_done_event_SEM = multiprocessing.Event()
    row_event_collector_validator_SEM = multiprocessing.Event()

    pipe_from_extractor_head_SEM, pipe_from_extractor_tail_SEM = multiprocessing.Pipe()

    pipe_from_collector_head_SEM, pipe_from_collector_tail_SEM = multiprocessing.Pipe()

    row_event_collector_validator_SEM = multiprocessing.Event()

    type_name = '_SEM'

    cell_extractor_SEM = Cell_extractor(type_name, cell_queue_SEM, partial_cell_queue_SEM, SEM_path, 
                                           extractor_done_event_SEM, row_done_event_SEM, pipe_from_extractor_head_SEM)
    cell_collector_SEM = Cell_collector(type_name, cell_queue_SEM, sorted_cell_queue_SEM,
                                           extractor_done_event_SEM, merger_done_event_SEM, 
                                           row_done_event_SEM, pipe_from_extractor_tail_SEM, collector_done_event_SEM, 
                                           pipe_from_collector_head_SEM, row_event_collector_validator_SEM)
    cell_merger_SEM = Cell_merger(type_name, partial_cell_queue_SEM, cell_queue_SEM, 
                                     extractor_done_event_SEM, merger_done_event_SEM)

    
    cell_validation = Cell_validation(sorted_cell_queue_layout, sorted_cell_queue_SEM, 
                                      collector_done_event_layout, collector_done_event_SEM,
                                      pipe_from_collector_tail_layout, pipe_from_collector_tail_SEM, 
                                      row_event_collector_validator_layout, row_event_collector_validator_SEM)

    cell_extractor_SEM.start()
    cell_collector_SEM.start()
    cell_merger_SEM.start()
    cell_extractor_layout.start()
    cell_collector_layout.start()
    cell_merger_layout.start()
    cell_validation.start()
    
    cell_extractor_SEM.join()
    cell_collector_SEM.join()
    cell_merger_SEM.join()
    cell_extractor_layout.join()
    cell_collector_layout.join()
    cell_merger_SEM.join()

    cell_validation.join()


    
    
    



 Cell_collector-9_layout Child thread exiting

 Cell_collector-12_SEM Child thread exiting
 Cell_collector-12_SEM Parent thread exiting



In [ ]:
# from PIL import Image
# import numpy as np
# import os
# import matplotlib.pyplot as plt

# # Open the image and load only the metadata (without loading the image data)
# SEM_name = "36.png"  # Replace with your image path 
# image_path = os.path.join( os.getcwd(), 'images', SEM_name)
# image = Image.open(image_path)

# # Define the region of interest (ROI)
# x, y, width, height = 100, 100, 200, 200  # Define your own coordinates and dimensions

# # Load the region of interest into memory
# roi = image.crop((x, y, x+width, y+height))

# # Convert the PIL Image to numpy array
# roi_array = np.array(roi)

# plt.imshow(roi)

In [ ]:
# print(type(roi))
# print(roi_array.shape)
# x = roi_array[:,:,0]
# plt.imshow(x)

# import cv2
# SEM_name = "36.png"  # Replace with your image path 
# image_path = os.path.join( os.getcwd(), 'images', SEM_name)
# roi_array = cv2.imread(image_path)
# print(type(roi_array))
# print(roi_array.shape)

In [ ]:
# SEM_name = "36.png"  # Replace with your image path 
# image_path = os.path.join( os.getcwd(), 'images', SEM_name)
# image = Image.open(image_path)
# x, y = image.size
# print(x,y)

x = []
y = []
x.append(1)
x.append(2)
y.append(3)
y.append(4)
x.append(y)

print(x)


[1, 2, [3, 4]]
